In [ ]:

import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from transformers import pipeline, DistilBertTokenizer, DistilBertForSequenceClassification

# Ensure necessary NLTK resources are downloaded
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
# @title processed dataset for risk and urgency levels


# Load the dataset
df = pd.read_csv("gpt-4.csv", engine='python', on_bad_lines='skip')
data = pd.DataFrame(df)

# Define more comprehensive keywords for risk and urgency levels
low_risk_keywords = ["mild", "better", "normal", "routine", "no symptoms"]
medium_risk_keywords = ["fever", "cough", "body aches", "headache", "fatigue", "chills"]
high_risk_keywords = ["difficulty breathing", "chest pain", "emergency", "severe", "critical", "intense", "unconscious", "high fever"]

non_urgent_keywords = ["check-up", "follow-up", "routine", "monitor"]
urgent_keywords = ["severe", "noticeable symptoms", "trouble breathing", "headache worsening", "nausea", "dizziness"]
critical_keywords = ["life-threatening", "ICU", "critical", "can't move", "unresponsive", "hospitalization"]

# Load the tokenizer for the DistilBERT model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
sentiment_model = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", tokenizer=tokenizer)

# Helper function to truncate long texts using the tokenizer
def truncate_text(text, max_length=512):
    inputs = tokenizer(text, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    return tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

# Function to classify risk, urgency, and sentiment (added more detailed classifications)
def classify_risk_urgency(conversation):
    # Handle empty or missing conversations
    if pd.isna(conversation) or conversation.strip() == "":
        return "Unknown", "Unknown", "Neutral"

    # Truncate the conversation to avoid token length issues
    conversation = truncate_text(conversation, max_length=512)

    # Classify risk level based on keyword frequency and intensity
    risk_score = 0
    if any(word in conversation.lower() for word in high_risk_keywords):
        risk_score += 3  # Higher score for critical keywords
    if any(word in conversation.lower() for word in medium_risk_keywords):
        risk_score += 2
    if any(word in conversation.lower() for word in low_risk_keywords):
        risk_score += 1

    if risk_score >= 5:
        risk = "High Risk"
    elif risk_score == 3 or risk_score == 4:
        risk = "Medium Risk"
    else:
        risk = "Low Risk"

    # Classify urgency level based on keyword matching
    urgency_score = 0
    if any(word in conversation.lower() for word in critical_keywords):
        urgency_score += 3
    if any(word in conversation.lower() for word in urgent_keywords):
        urgency_score += 2
    if any(word in conversation.lower() for word in non_urgent_keywords):
        urgency_score += 1

    if urgency_score >= 5:
        urgency = "Critical"
    elif urgency_score == 3 or urgency_score == 4:
        urgency = "Urgent"
    else:
        urgency = "Non-Urgent"

    # Perform sentiment analysis to aid with urgency classification
    sentiment_result = sentiment_model(conversation)
    sentiment = sentiment_result[0]['label'] if sentiment_result else "Neutral"

    return risk, urgency, sentiment

# Apply the classification function to the dataset
data[["Risk", "Urgency", "Sentiment"]] = data["conversation"].apply(
    lambda x: pd.Series(classify_risk_urgency(x))
)

# Save the labeled dataset to a new CSV file
data.to_csv("labeled_dataset_with_sentiment1.csv", index=False)

# Display the labeled dataset
print(data.head())


In [ ]:
pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib pandas


**processed dataset with assigned risk and urgency levels has been utilized for this task to schedule calendar events.**

**The top 10 high-risk patient data has been used as a sample to schedule meetings**

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# Step 1: Setup Google Calendar API
SERVICE_ACCOUNT_FILE = 'credentials.json'  # Path to your downloaded JSON credentials file
SCOPES = ['https://www.googleapis.com/auth/calendar']

credentials = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

# Step 2: Define a function to create a calendar event
def create_event(service, summary, description, start_time, end_time):
    event = {
        'summary': summary,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': 'Asia/Kolkata',  # IST Time Zone
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'Asia/Kolkata',  # IST Time Zone
        },
    }
    created_event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {created_event.get('htmlLink')}")

# Step 3: Load the processed dataset
data = pd.read_csv("labeled_dataset_with_sentiment.csv")

# Step 4: Filter and handle only the top 10 high-risk patients
high_risk_data = data[data['Risk'] == "High Risk"].head(10)

# Step 5: Iterate through the filtered dataset and create events
for index, row in high_risk_data.iterrows():
    risk = row['Risk']
    urgency = row['Urgency']
    patient_details = row['data']
    conversation = row['conversation']

    # Set meeting time based on urgency (IST times)
    if urgency == "Critical":
        start_time = "2025-01-24T10:00:00+05:30"  # Example start time
        end_time = "2025-01-24T10:30:00+05:30"    # Example end time
    elif urgency == "Urgent":
        start_time = "2025-01-24T11:00:00+05:30"
        end_time = "2025-01-24T11:30:00+05:30"
    else:
        start_time = "2025-01-24T12:00:00+05:30"
        end_time = "2025-01-24T12:30:00+05:30"

    # Event details
    event_summary = f"Follow-up Meeting with High-Risk Patient"
    event_description = f"""
    Patient Details: {patient_details}
    Conversation: {conversation}
    Risk Level: {risk}
    Urgency: {urgency}
    """

    # Create the calendar event
    create_event(service, event_summary, event_description, start_time, end_time)


Event created: https://www.google.com/calendar/event?eid=YWt2amh2aTUxbzc5YWppaGp1a2cxc2g5dW8gY2FsZW5kYXItaW50ZWdyYXRpb24tc2VydmljZUBzaW5nbGUtY29ucXVlc3QtNDQ4NzA4LWI2LmlhbS5nc2VydmljZWFjY291bnQuY29t
Event created: https://www.google.com/calendar/event?eid=dGw5ZzUwaTkzZWlobXBzN21iNGEyNWVzZjQgY2FsZW5kYXItaW50ZWdyYXRpb24tc2VydmljZUBzaW5nbGUtY29ucXVlc3QtNDQ4NzA4LWI2LmlhbS5nc2VydmljZWFjY291bnQuY29t
Event created: https://www.google.com/calendar/event?eid=Y25qYnM1NmszbGEyZGk0cXI4azI1dDl2NjAgY2FsZW5kYXItaW50ZWdyYXRpb24tc2VydmljZUBzaW5nbGUtY29ucXVlc3QtNDQ4NzA4LWI2LmlhbS5nc2VydmljZWFjY291bnQuY29t
Event created: https://www.google.com/calendar/event?eid=ZmY3ZXZ0N2EydmdzZm1hY3RwZ3BhcjJpaGcgY2FsZW5kYXItaW50ZWdyYXRpb24tc2VydmljZUBzaW5nbGUtY29ucXVlc3QtNDQ4NzA4LWI2LmlhbS5nc2VydmljZWFjY291bnQuY29t
Event created: https://www.google.com/calendar/event?eid=czRxdTJvaXMyNWcxYXFsbzVqaWk5ZHRvc3MgY2FsZW5kYXItaW50ZWdyYXRpb24tc2VydmljZUBzaW5nbGUtY29ucXVlc3QtNDQ4NzA4LWI2LmlhbS5nc2VydmljZWFjY291bnQuY29t
Event crea

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# Step 1: Setup Google Calendar API
SERVICE_ACCOUNT_FILE = 'credentials.json'  # Path to your JSON credentials file
SCOPES = ['https://www.googleapis.com/auth/calendar']

# Authenticate the service account
credentials = Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('calendar', 'v3', credentials=credentials)

# Step 2: Define a function to create a calendar event with debugging
def create_event(service, summary, description, start_time, end_time):
    event = {
        'summary': summary,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': 'Asia/Kolkata',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'Asia/Kolkata',
        },
    }
    print("Creating event with payload:", event)  # Debug payload
    try:
        created_event = service.events().insert(calendarId='primary', body=event).execute()
        print(f"Event successfully created: {created_event.get('htmlLink')}")
    except Exception as e:
        print(f"Error creating event: {e}")

# Step 3: Verify available calendars
print("Fetching available calendars...")
calendars = service.calendarList().list().execute()
for calendar in calendars['items']:
    print(f"Calendar ID: {calendar['id']} - Summary: {calendar['summary']}")

# Step 4: Load the processed dataset
data = pd.read_csv("labeled_dataset_with_sentiment.csv")

# Step 5: Filter and handle only the top 10 high-risk patients
high_risk_data = data[data['Risk'] == "High Risk"].head(10)

# Step 6: Iterate through the filtered dataset and create events
for index, row in high_risk_data.iterrows():
    risk = row['Risk']
    urgency = row['Urgency']
    patient_details = row['data']
    conversation = row['conversation']

    # Set meeting time based on urgency
    if urgency == "Critical":
        start_time = "2025-01-24T10:00:00+05:30"  # IST time zone
        end_time = "2025-01-24T10:30:00+05:30"
    elif urgency == "Urgent":
        start_time = "2025-01-24T11:00:00+05:30"
        end_time = "2025-01-24T11:30:00+05:30"
    else:
        start_time = "2025-01-24T12:00:00+05:30"
        end_time = "2025-01-24T12:30:00+05:30"

    # Event details
    event_summary = f"Follow-up Meeting with High-Risk Patient"
    event_description = f"""
    Patient Details: {patient_details}
    Conversation: {conversation}
    Risk Level: {risk}
    Urgency: {urgency}
    """

    # Create the calendar event
    create_event(service, event_summary, event_description, start_time, end_time)


Fetching available calendars...
Creating event with payload: {'summary': 'Follow-up Meeting with High-Risk Patient', 'description': "\n    Patient Details: This 60-year-old male was hospitalized due to moderate ARDS from COVID-19 with symptoms of fever, dry cough, and dyspnea. We encountered several difficulties during physical therapy on the acute ward. First, any change of position or deep breathing triggered coughing attacks that induced oxygen desaturation and dyspnea. To avoid rapid deterioration and respiratory failure, we instructed and performed position changes very slowly and step-by-step. In this way, a position change to the 135° prone position () took around 30 minutes. This approach was well tolerated and increased oxygen saturation, for example, on day 5 with 6 L/min of oxygen from 93% to 97%. Second, we had to adapt the breathing exercises to avoid prolonged coughing and oxygen desaturation. Accordingly, we instructed the patient to stop every deep breath before the nee

**steps to set up the Google Cloud Console for the Google Calendar API:**


### **Step 1: Create a Google Cloud Project**
1. Go to the Google Cloud Console: [Google Cloud Console](https://console.cloud.google.com/).
2. Sign in with your Google account.
3. Create a new project:
   - Click on the "Select a Project" dropdown in the top-left corner.
   - Click "New Project."
   - Enter a Project Name (e.g., "Google Calendar Integration").
   - Click "Create."



### **Step 2: Enable the Google Calendar API**
1. In the Google Cloud Console, go to **APIs & Services > Library**.
2. In the search bar, type "Google Calendar API."
3. Click on the **Google Calendar API** result.
4. Click **Enable**.



### **Step 3: Create Credentials for the API**
1. Go to **APIs & Services > Credentials**.
2. Click **Create Credentials > Service Account**.
3. Fill in the details:
   - **Service Account Name**: Enter a name (e.g., "Calendar Integration Service").
   - **Service Account ID**: Automatically generated.
   - **Description**: Optional.
4. Click **Create and Continue**.
5. Assign roles:
   - Select **Basic > Editor** or **Owner** role.
6. Click **Continue** and then **Done**.



### **Step 4: Download the Credentials File**
1. After creating the service account, go back to the **Credentials** page.
2. Locate your service account under the **Service Accounts** section.
3. Click the **Edit icon** (pencil) next to the service account.
4. Go to the **Keys** tab.
5. Click **Add Key > Create New Key**.
6. Select **JSON** as the key type.
7. Download the **credentials.json** file. Keep this file safe, as it contains sensitive information.


### **Step 5: Share Your Calendar with the Service Account**
1. Open **Google Calendar** and sign in.
2. Select the calendar you want to use for this integration.
3. Go to **Settings and Sharing** for that calendar.
4. Under **Share with specific people**, click **Add people**.
5. Enter the **Service Account Email** from your credentials.json file (it ends with `@<project-id>.iam.gserviceaccount.com`).
6. Set the permissions to **Make changes to events**.
7. Click **Send**.
